In [ ]:
import pandas as pd
import numpy as np

Upload and check data 

In [ ]:
sales = pd.read_csv('product_sales.csv')

Checking states spelling and duplicates 

In [ ]:
unique_values = sales['state'].unique()
sorted_unique_values = sorted(unique_values)

Dealing with years as customer row

In [ ]:
from datetime import datetime
company_founding_date = datetime(1984, 1, 1)
current_date = datetime.today()
years_since_founding = current_date.year - company_founding_date.year
print("Years since company founding:", years_since_founding)
values_exceeding_years_since_founding = sales[sales['years_as_customer'] > years_since_founding]
count_exceeding_years_since_founding = len(values_exceeding_years_since_founding)

In [ ]:
sales.loc[sales['years_as_customer'] > 39, 'years_as_customer'] = sales['years_as_customer'].mean()

In [ ]:
sales.isnull().sum()

Dealing with sales methods 

In [ ]:
# count number of unique values for sales method
num_customer = sales['customer_id'].nunique()
num_sales_method = sales['sales_method'].nunique()

#identity which sales methods are misspelt
sales['sales_method'].unique()

replacement_dict = {
    'em + call': 'Email + Call',
    'email': 'Email',
}

#replace misspelt words
sales['sales_method'] = sales['sales_method'].replace(replacement_dict)

# Task 1 - How many customers were there for each approach

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#count number of customers per method
num_methods = sales['sales_method'].value_counts()
num_methods

In [ ]:
#visualise customers approached for each sales method 
plt.bar(num_methods.index, num_methods.values)
plt.xlabel('Sales Method')
plt.ylabel('Number of Customers')
plt.title('Customer Count by Sales Method')
plt.show()

# Task 2 - What does the spread of the revenue look like overall? And for each method?

In [ ]:
# remove missing values 
sales.dropna(inplace=True)

In [ ]:
import seaborn as sns

The spread of revenue shows that there are more buyers 

In [ ]:
#visualise the spread of revenue 
plt.figure(figsize=(8, 6))
sns.histplot(data=sales, x='revenue', kde=True, color='skyblue')
plt.xlabel('Revenue')
plt.ylabel('Frequency')
plt.title('Distribution of Revenue')
plt.show()

Email + call = large revenue spread and more effective producer of revenue

call = smaller spread, but less prodcutive source of revenue



In [ ]:
#spread of revenue for each method 
plt.figure(figsize=(10, 6))
sns.boxplot(data=sales, x='sales_method', y='revenue', palette='Set3')
plt.xlabel('Sales Method')
plt.ylabel('Revenue')
plt.title('Spread of Revenue by Sales Method')
plt.show()

In [ ]:
Spread of revenue per state

In [ ]:
revenue_by_state = sales.groupby('state')['revenue'].sum().reset_index()

revenue_by_state = revenue_by_state.sort_values(by='revenue', ascending=False)

plt.figure(figsize=(10, 6))
plt.bar(revenue_by_state['state'], revenue_by_state['revenue'])
plt.title('Total Revenue per State')
plt.xlabel('State')
plt.ylabel('Total Revenue')
plt.xticks(rotation=45, ha='right')  # Rotate state labels for better readability

plt.show()

In [ ]:
revenue_by_state.head(10)

site visits correlation 

In [ ]:
correlation = sales['nb_site_visits'].corr(sales['revenue'])

print("Correlation between nb_site_visits and revenue:", correlation)

In [ ]:
plt.figure(figsize=(10, 6))
sns.regplot(x='nb_site_visits', y='revenue', data=sales, scatter_kws={'alpha': 0.5})
plt.title('Regression Plot of Site Visits vs. Revenue')
plt.xlabel('Number of Site Visits')
plt.ylabel('Revenue')

plt.show()


# Task 3 - Was there any difference in revenue over time for each of the methods?

In [ ]:
sales['week'].unique()

Email + call = showed the most amout of revenue progress in the 6 week assessment, whereas just calling showed relatively less. 



In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=sales, x='week', y='revenue', hue='sales_method', palette='Set3')
plt.xlabel('Week')
plt.ylabel('Revenue')
plt.title('Revenue Over Time for Different Sales Methods')
plt.legend(title='Sales Method')
plt.show()

In [ ]:
correlation = sales['years_as_customer'].corr(sales['revenue'])

print("Correlation between years_as_customer and revenue:", correlation)

In [ ]:
plt.figure(figsize=(10, 6))
sns.regplot(x='years_as_customer', y='revenue', data=sales, scatter_kws={'alpha':0.5})
plt.title('Regression Plot of Years as Customer vs. Revenue')
plt.xlabel('Years as Customer')
plt.ylabel('Revenue')
plt.grid(True)

plt.show()

# Task 4 - Based on the data, which method would you recommend we continue to use? 
Some of these methods take more time from the team so they may not be the best for us to use if the results are similar.

In [ ]:
total_revenue_per_method = sales.groupby('sales_method')['revenue'].sum()
total_revenue = sales['revenue'].sum()

In [ ]:
percentage_per_method = (total_revenue_per_method / total_revenue) * 100

In [ ]:
percentage_per_method

In [ ]:
total_revenue_vis = sales.groupby('sales_method')['revenue'].sum().reset_index()
total_revenue_vis

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(total_revenue_vis['sales_method'], total_revenue_vis['revenue'], color='skyblue')
plt.xlabel('Sales Method')
plt.ylabel('Total Revenue')
plt.title('Total Revenue for Each Sales Method')
plt.show()

The total revenue from emailing customers 

In [ ]:
method_to_numeric = {
    'Call': 30,
    'Email': 0,
    'Email + Call': 10
}

In [ ]:
revenue_no_null['Time_Spent_Customer'] = revenue_no_null['sales_method'].map(method_to_numeric)
revenue_no_null.head()

In [ ]:
revenue_no_null['revenue'].sum()

In [ ]:
total_customers = revenue_no_null.groupby('sales_method').size().reset_index(name='TotalCustomers')
total_customers

In [ ]:
avg_revenue_per_customer = revenue_no_null.groupby('sales_method')['revenue'].mean().reset_index(name='AvgRevenuePerCustomer')


In [ ]:
result_df = pd.merge(total_customers, avg_revenue_per_customer, on='sales_method')
result_df


In [ ]:
result_df['total_revenue'] = result_df['TotalCustomers'] * result_df['AvgRevenuePerCustomer']
result_df

In [ ]:
result_df['sales_method'] = result_df['sales_method'].replace('Call', 'Email + Call Projected')

In [ ]:
result_df['total_revenue'] = result_df['total_revenue'].replace(227563.49, 878036.54)

In [ ]:
result_df['total_revenue'].sum()

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(result_df['sales_method'], result_df['total_revenue'])
plt.xlabel('Sales Method')
plt.ylabel('total_revenue')
plt.title('Email + Call Potential if Resource Was Redirected')
plt.show()

In [ ]:
total_cust = result_df[result_df['sales_method'] == 'Email + Call']['AvgRevenuePerCustomer'].values[0]
avg_rev = result_df[result_df['sales_method'] == 'Call']['TotalCustomers'].values[0]
result = total_cust * avg_rev
result

In [ ]:
total_time_spent_per_method = revenue_no_null.groupby('sales_method')['Time_Spent_Customer'].sum().reset_index()
total_time_spent_per_method

In [ ]:
total_time_rev = total_time_spent_per_method.merge(total_revenue_vis, on='sales_method')
total_time_rev

In [ ]:
total_time_rev['percentage_rev_per_method'] = (total_time_rev['revenue'] / total_time_rev['revenue'].sum()) * 100

In [ ]:
total_time_rev['hours_spent_on_customer'] =  total_time_rev['Time_Spent_Customer'] / 60

In [ ]:
total_time_rev = total_time_rev.drop('Time_Spent_Customer', axis=1 )

In [ ]:
total_time_rev = total_time_rev.round(2)

In [ ]:
plt.figure(figsize=(10, 6))
for method, group in total_time_rev.groupby('sales_method'):
    plt.scatter(group['hours_spent_on_customer'], group['percentage_rev_per_method'], label=method, s=100)
    

plt.xlabel('Hours Spent on Customer')
plt.ylabel('Percentage of Revenue per Customer')
plt.title('Percentage of Revenue per Customer vs Hours Spent on Customer')
plt.legend()
plt.show()

## ✅ When you have finished...
-  Publish your Workspace using the option on the left
-  Check the published version of your report:
	-  Can you see everything you want us to grade?
    -  Are all the graphics visible?
-  Review the grading rubric. Have you included everything that will be graded?
-  Head back to the [Certification Dashboard](https://app.datacamp.com/certification) to submit your practical exam report and record your presentation